### Fine tune per signal LSTMs

Train LSTMs by initializing them with the best LSTM from the other hospital data set.

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
from phase.embedding import *

PATH = os.path.expanduser("~/upstream_embedding/")

def get_datapath(PATH,feat,hosp_data,task):
    DATAPATH  = '{}data/min5_data/{}minimum5/hospital_{}/proc/'.format(PATH,feat,hosp_data)
    if task == "hypox":
        DATAPATH  = '{}data/desat_bool92_5_nodesat/hospital_{}/proc/'.format(PATH,hosp_data)
    elif task == "hypoc":
        DATAPATH  = '{}data/etco235/hospital_{}/proc/'.format(PATH,hosp_data)
    elif task == "hypot":
        DATAPATH  = '{}data/nibpm60/hospital_{}/proc/'.format(PATH,hosp_data)
    return(DATAPATH)

for task in ["nextfive", "minimum5", "hypox", "hypot", "hypoc"]:
    for hosp_data in [0,1]:
        print("[Progress] Current hospital: {}".format(hosp_data))
        for i in range(0,len(X_lst)):
            
            # Select opposite hospital to initialize
            if hosp_data == 0:
                hosp_source = 1
            elif hosp_data == 1:
                hosp_source = 0
            
            # Current feature
            feat = X_lst[i]

            # Load data
            DATAPATH  = get_datapath(PATH,feat,hosp_data,task)
            train_x, train_y, val_x, val_y = load_train_val_data(DATAPATH, task, feat)

            # Load model and initialize new model with these weights
            source_mpath = "{}models/200epochs/{}/hospital_{}/".format(PATH,task,hosp_source)
            min_mod = load_min_model(source_mpath, feat)
            model, mod_name, epoch_num = create_model(train_y.shape[1], min_mod=min_mod, 
                                                      fine_tune=True, task=task)

            # Set up model path
            curr_task = "feat:{}_task:{}_hosp:{}to{}".format(feat,task,hosp_source,hosp_data)
            MODDIR    = "{}models/200epochs/{}/".format(PATH,task)
            MODDIR    += "hospital_{}/{}_{}/".format(hosp_data,curr_task,mod_name)
            print("MODDIR {}".format(MODDIR))

            if not os.path.exists(MODDIR): os.makedirs(MODDIR)

            # Train model
            train_model(model, mod_name, train_x, train_y, val_x, val_y, 
                        MODDIR, epoch_num, early_stopping_rounds=200)